# Validate configuration for reproducing "Revisiting TCP Congestion Control Throughput Models & Fairness Properties at Scale"

## Set up your FABRIC environment


In [55]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
fablib.show_config()

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,073ee843-2310-45bd-a01f-a15d808827dc
Bastion Username,ffund_0041777137
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,073ee843-2310-45bd-a01f-a15d808827dc
Bastion Username,ffund_0041777137
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


In [56]:
!chmod 600 {fablib.get_bastion_key_filename()}
!chmod 600 {fablib.get_default_slice_private_key_file()}

## Get slice details

Put your slice name and the number of endpoints in the following cell:

In [57]:
slice_name = "bottleneck-10-test"
n_endpoints = 10

Then, load your slice details into the environment.slice = fablib.new_slice(name=slice_name)

In [58]:
slice = fablib.get_slice(name=slice_name)

In [59]:
sender_nodes = [slice.get_node(name='sender-' + str(i))  for i in range(n_endpoints)]
receiver_nodes = [slice.get_node(name='receiver-' + str(i))  for i in range(n_endpoints)]

In [60]:
router_node = slice.get_node(name='router')
router_ingress_iface = router_node.get_interface(network_name = "link-sender")
router_egress_iface  = router_node.get_interface(network_name = "link-receiver")
router_egress_name = router_egress_iface.get_device_name()

## Check network capacity

We should get close to 25 Gbps (probably more like 23 Gbps because of protocol overhead) through the router.

First, make sure there is no rate limiting at the router - don't worry if you get an error message, this just means there was already no rate limiting.

In [61]:
router_node.execute("sudo tc qdisc del dev " + router_egress_name + " root")

 Error: Cannot delete qdisc with handle of zero.
 

('', 'Error: Cannot delete qdisc with handle of zero.\n')

Now start the `iperf3` flows:

In [62]:
for n in receiver_nodes:
    n.execute("iperf3 -s -1 -f g -D --logfile validate.dat")

In [63]:
import time
for n in sender_nodes:
    n.execute_thread("iperf3 -t 60 -P 10 -c 10.10.2.1$(hostname | cut -d'-' -f7)")
time.sleep(65)

In [64]:
tputs = []
for n in receiver_nodes:
    (nout, nerr) = n.execute("tail --lines=2 validate.dat | grep receiver | awk '{print $6}'")
    tputs.append(float(nout.strip()))
print("Sum throughput is: %f Gbps" % sum(tputs))

2.18
1.64
1.66
3.85
1.63
2.47
1.38
1.41
4.02
3.10
Sum throughput is: 23.340000 Gbps


A single pair of endpoints should be able to get about 10 Gbps, possibly higher.

In [65]:
receiver_nodes[0].execute("iperf3 -s -1 -f g -D")
sender_nodes[0].execute("iperf3 -t 60 -i 60 -P 10 -c 10.10.2.1$(hostname | cut -d'-' -f7)")

Connecting to host 10.10.2.10, port 5201
[  4] local 10.10.1.10 port 50670 connected to 10.10.2.10 port 5201
[  6] local 10.10.1.10 port 50672 connected to 10.10.2.10 port 5201
[  8] local 10.10.1.10 port 50674 connected to 10.10.2.10 port 5201
[ 10] local 10.10.1.10 port 50676 connected to 10.10.2.10 port 5201
[ 12] local 10.10.1.10 port 50678 connected to 10.10.2.10 port 5201
[ 14] local 10.10.1.10 port 50680 connected to 10.10.2.10 port 5201
[ 16] local 10.10.1.10 port 50682 connected to 10.10.2.10 port 5201
[ 18] local 10.10.1.10 port 50684 connected to 10.10.2.10 port 5201
[ 20] local 10.10.1.10 port 50686 connected to 10.10.2.10 port 5201
[ 22] local 10.10.1.10 port 50688 connected to 10.10.2.10 port 5201
[ ID] Interval           Transfer     Bandwidth       Retr  Cwnd
[  4]   0.00-60.00  sec  9.62 GBytes  1.38 Gbits/sec  2680    288 KBytes       
[  6]   0.00-60.00  sec  9.60 GBytes  1.37 Gbits/sec  2534    328 KBytes       
[  8]   0.00-60.00  sec  9.60 GBytes  1.37 Gbits/sec  

('Connecting to host 10.10.2.10, port 5201\n[  4] local 10.10.1.10 port 50670 connected to 10.10.2.10 port 5201\n[  6] local 10.10.1.10 port 50672 connected to 10.10.2.10 port 5201\n[  8] local 10.10.1.10 port 50674 connected to 10.10.2.10 port 5201\n[ 10] local 10.10.1.10 port 50676 connected to 10.10.2.10 port 5201\n[ 12] local 10.10.1.10 port 50678 connected to 10.10.2.10 port 5201\n[ 14] local 10.10.1.10 port 50680 connected to 10.10.2.10 port 5201\n[ 16] local 10.10.1.10 port 50682 connected to 10.10.2.10 port 5201\n[ 18] local 10.10.1.10 port 50684 connected to 10.10.2.10 port 5201\n[ 20] local 10.10.1.10 port 50686 connected to 10.10.2.10 port 5201\n[ 22] local 10.10.1.10 port 50688 connected to 10.10.2.10 port 5201\n[ ID] Interval           Transfer     Bandwidth       Retr  Cwnd\n[  4]   0.00-60.00  sec  9.62 GBytes  1.38 Gbits/sec  2680    288 KBytes       \n[  6]   0.00-60.00  sec  9.60 GBytes  1.37 Gbits/sec  2534    328 KBytes       \n[  8]   0.00-60.00  sec  9.60 GBytes  

## Check network delay

We should get sub-ms latency across the router.

In [66]:
for n in sender_nodes:
    n.execute("ping -c 5 10.10.2.1$(hostname | cut -d'-' -f7) | grep rtt")

rtt min/avg/max/mdev = 0.129/0.157/0.214/0.033 ms
rtt min/avg/max/mdev = 0.155/0.193/0.310/0.058 ms
rtt min/avg/max/mdev = 0.114/0.161/0.303/0.073 ms
rtt min/avg/max/mdev = 0.134/0.172/0.273/0.053 ms
rtt min/avg/max/mdev = 0.154/0.201/0.358/0.080 ms
rtt min/avg/max/mdev = 0.119/0.172/0.309/0.069 ms
rtt min/avg/max/mdev = 0.131/0.179/0.331/0.077 ms
rtt min/avg/max/mdev = 0.123/0.161/0.304/0.072 ms
rtt min/avg/max/mdev = 0.124/0.171/0.267/0.051 ms
rtt min/avg/max/mdev = 0.128/0.197/0.397/0.101 ms
